# Extract transactions from statements 13-16

This notebook implements template extraction for `onlineStatement (13).pdf` through `onlineStatement (16).pdf`.
It extracts these fields into an array of dictionaries: `transaction_date`, `post_date`, `description`, `category`, and `amount`.

In [ ]:
import pdfplumber
import re
import csv

# Your statement files inside the 'data/' folder
pdf_files = [
    "data/onlineStatement (13).pdf",
    "data/onlineStatement (14).pdf",
    "data/onlineStatement (15).pdf",
    "data/onlineStatement (16).pdf"
]


output_csv = "data/cibc_transactions.csv"


In [ ]:
transaction_pattern = re.compile(
    r"^([A-Za-z]{3}\s+\d{1,2})\s+([A-Za-z]{3}\s+\d{1,2})\s+(.*?)\s+([A-Za-z& ]+)\s+(\d+\.\d{2})$"
)

transactions = []


In [ ]:
for pdf in pdf_files:
    with pdfplumber.open(pdf) as pdf_obj:
        for page in pdf_obj.pages:
            text = page.extract_text()
            if not text:
                continue
            for line in text.split("\n"):
                match = transaction_pattern.match(line.strip())
                if match:
                    trans_date, post_date, description, category, amount = match.groups()
                    transactions.append({
                        "Transaction Date": trans_date,
                        "Post Date": post_date,
                        "Description": description.strip(),
                        "Category": category.strip(),
                        "Amount": amount
                    })

print(f"Extracted {len(transactions)} transactions")


Extracted 171 transactions


In [ ]:
with open(output_csv, mode="w", newline="", encoding="utf-8") as csvfile:
    fieldnames = ["Transaction Date", "Post Date", "Description", "Category", "Amount"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(transactions)

print(f" Transactions saved to {output_csv}")


 Transactions saved to data/cibc_transactions.csv


In [ ]:
import pandas as pd

df = pd.read_csv(output_csv)
df.head(10)  # Show first 10 rows


,Transaction Date,Post Date,Description,Category,Amount
0,Aug 25,Aug 26,PAYMENT THANK YOU/PAIEMENT,MERCI,79.61
1,Sep 02,Sep 03,PAYMENT THANK YOU/PAIEMENT,MERCI,30.54
2,Sep 04,Sep 08,PAYMENT THANK YOU/PAIEMENT,MERCI,33.25
3,Sep 05,Sep 09,PAYMENT THANK YOU/PAIEMENT,MERCI,1.60
4,Sep 09,Sep 10,PAYMENT THANK YOU/PAIEMENT,MERCI,5.18
5,Aug 17,Aug 18,MY,SPICE HOUSE WINNIPEG MB Retail and Grocery,11.00
6,Aug 17,Aug 18,REAL CDN. SUPERSTORE #,WINNIPEG MB Retail and Grocery,22.37
7,Aug 20,Aug 21,MPI,BISON SERVICE CENTRE WINNIPEG MB Professional ...,25.00
8,Aug 20,Aug 24,SOBEYS #5037,WINNIPEG MB Retail and Grocery,15.76
9,Aug 22,Aug 24,TIM HORTONS #8152,WINNIPEG MB Restaurants,1.98


In [ ]:
df = pd.read_csv("data/cibc_transactions.csv")
df["Transaction Date"] = pd.to_datetime(df["Transaction Date"] + " 2020", format="%b %d %Y")
df["Month"] = df["Transaction Date"].dt.strftime("%B %Y")


In [ ]:
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

def plot_month_spend(month):
    # Filter data for selected month
    month_data = df[df["Month"] == month]

    if month_data.empty:
        print("No transactions for this month.")
        return

    # Convert Amount to numeric
    month_data["Amount"] = pd.to_numeric(month_data["Amount"], errors="coerce")

    # Group by category and sum
    spend_summary = month_data.groupby("Category")["Amount"].sum().sort_values(ascending=False)

    # Plot pie chart
    plt.figure(figsize=(7,7))
    plt.pie(
        spend_summary,
        labels=spend_summary.index,
        autopct='%1.1f%%',     # show percentages
        startangle=90,
        counterclock=False
    )
    total_spend = spend_summary.sum()
    plt.title(f"Spending Breakdown - {month}\nTotal: ${total_spend:,.2f}", fontsize=14)
    plt.tight_layout()
    plt.show()

# Dropdown for selecting month
widgets.interact(
    plot_month_spend,
    month=widgets.Dropdown(
        options=sorted(df["Month"].unique()),
        description="Select Month:",
        style={'description_width': 'initial'}
    )
);


interactive(children=(Dropdown(description='Select Month:', options=('August 2020', 'December 2020', 'November…

In [ ]:
import pandas as pd


df = pd.read_csv("data/cibc_transactions.csv")

df.head()


,Transaction Date,Post Date,Description,Category,Amount
0,Aug 25,Aug 26,PAYMENT THANK YOU/PAIEMENT,MERCI,79.61
1,Sep 02,Sep 03,PAYMENT THANK YOU/PAIEMENT,MERCI,30.54
2,Sep 04,Sep 08,PAYMENT THANK YOU/PAIEMENT,MERCI,33.25
3,Sep 05,Sep 09,PAYMENT THANK YOU/PAIEMENT,MERCI,1.60
4,Sep 09,Sep 10,PAYMENT THANK YOU/PAIEMENT,MERCI,5.18


In [ ]:
df["Amount"] = pd.to_numeric(df["Amount"], errors="coerce")


df["Transaction Date"] = pd.to_datetime(df["Transaction Date"], errors="coerce")


df["DayOfWeek"] = df["Transaction Date"].dt.dayofweek
df["Day"] = df["Transaction Date"].dt.day
df["Month"] = df["Transaction Date"].dt.month


df["Merchant"] = df["Description"].str.split().str[0]


df_encoded = pd.get_dummies(df[["Category", "Merchant", "DayOfWeek", "Day", "Month"]])

X = df_encoded
y = df["Amount"]


C:\Users\udugr\AppData\Local\Temp\ipykernel_71860\2086970872.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Transaction Date"] = pd.to_datetime(df["Transaction Date"], errors="coerce")


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

X_train = X_train.replace([np.inf, -np.inf], np.nan).fillna(0)
X_test = X_test.replace([np.inf, -np.inf], np.nan).fillna(0)


y_train = y_train.loc[X_train.index]
y_test = y_test.loc[X_test.index]

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)


mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f" Model trained successfully!")
print(f"Mean Squared Error: {mse:.2f}")
print(f"R² Score: {r2:.2f}")
print(f"Training samples used: {len(X_train)}, Test samples: {len(X_test)}")


 Model trained successfully!
Mean Squared Error: 417.78
R² Score: 0.33
Training samples used: 136, Test samples: 35


In [ ]:
# Compare actual vs predicted
comparison = pd.DataFrame({"Actual": y_test.values, "Predicted": y_pred})
comparison.head(10)


,Actual,Predicted
0,8.95,12.040000
1,30.98,35.659601
2,6.49,3.880000
3,27.10,28.613333
4,29.65,24.004444
5,1.33,3.880000
6,12.86,10.215000
7,2.23,2.740000
8,20.02,35.659601
9,2.26,10.215000


In [ ]:

X = df["Description"]
y = df["Category"]


In [ ]:
from sklearn.model_selection import train_test_split

counts = y.value_counts()
mask = y.isin(counts[counts > 1].index)
X = X[mask]
y = y[mask]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF vectorization
vectorizer = TfidfVectorizer(stop_words="english", ngram_range=(1,2), max_features=500)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

clf = RandomForestClassifier(random_state=42)
clf.fit(X_train_vec, y_train)

y_pred = clf.predict(X_test_vec)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 1.00

Classification Report:
                                      precision    recall  f1-score   support

                              MERCI       1.00      1.00      1.00         7
 RESTAURANT WINNIPEG MB Restaurants       1.00      1.00      1.00         2
SUM KINGDOM WINNIPEG MB Restaurants       1.00      1.00      1.00         1
   WINNIPEG MB Health and Education       1.00      1.00      1.00         4
            WINNIPEG MB Restaurants       1.00      1.00      1.00        13
     WINNIPEG MB Retail and Grocery       1.00      1.00      1.00         5
            Winnipeg MB Restaurants       1.00      1.00      1.00         1

                           accuracy                           1.00        33
                          macro avg       1.00      1.00      1.00        33
                       weighted avg       1.00      1.00      1.00        33



In [ ]:

sample = ["Tim Hortons", "Shoppers Drug Mart", "Burger King", "Pizza Pizza", "Dollarama"]
sample_vec = vectorizer.transform(sample)
preds = clf.predict(sample_vec)

for desc, cat in zip(sample, preds):
    print(f"{desc} → {cat}")


Tim Hortons → WINNIPEG MB Restaurants
Shoppers Drug Mart → SPICE HOUSE WINNIPEG MB Retail and Grocery
Burger King → SPICE HOUSE WINNIPEG MB Retail and Grocery
Pizza Pizza → WINNIPEG MB Restaurants
Dollarama → SPICE HOUSE WINNIPEG MB Retail and Grocery
